# Reusing the Data Access Object (DAO) Code in Multiple Python Projects

Reusing the Data Access Object (DAO) code in multiple Python projects can be achieved by properly abstracting and packaging the DAO layer. Here’s how you can make your DAO reusable:

## 1. Abstract DAO Code into a Standalone Package

### Directory Structure for the DAO Package

You need to refactor your DAO code into a standalone package. Here's an example of what that might look like:


In [ ]:
my_dao_package/
│
├── my_dao/
│   ├── __init__.py
│   ├── base_dao.py      # Base DAO class
│   ├── user_dao.py      # User DAO with CRUD operations
│   └── session.py       # Session management
│
├── tests/               # Unit tests for your DAOs
│   ├── __init__.py
│   └── test_user_dao.py
│
├── setup.py             # Package setup script
└── README.md            # Project description


## Example Code

### `my_dao/base_dao.py`

In [ ]:
class BaseDAO:
    def __init__(self, session):
        self.session = session

    def add(self, entity):
        self.session.add(entity)
        self.session.commit()

    def delete(self, entity):
        self.session.delete(entity)
        self.session.commit()


### `my_dao/user_dao.py`

In [ ]:
from .base_dao import BaseDAO
from .models import User

class UserDAO(BaseDAO):
    def get_user_by_id(self, user_id):
        return self.session.query(User).filter(User.id == user_id).first()

    def create_user(self, user):
        self.add(user)


### `my_dao/session.py`

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./test.db"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def get_session():
    return SessionLocal()


### `setup.py`

In [ ]:
from setuptools import find_packages, setup

setup(
    name='my_dao_package',
    version='0.1.0',
    packages=find_packages(),
    install_requires=[
        'SQLAlchemy==1.4.22',
    ],
    author="Your Name",
    author_email="your.email@example.com",
    description="A reusable DAO package for FastAPI",
    long_description=open('README.md').read(),
    long_description_content_type='text/markdown',
    url="https://github.com/yourusername/my_dao_package",
)


## 2. Publish the Package

### Local Installation

If you're sharing the package only within your organization, you can install it locally. Here's how to do it:

1. **Create a Distribution Package**  
   Use `setuptools` to create a distribution package. You can do this by running:

   ```bash
   python setup.py sdist bdist_wheel


In [ ]:
pip install -e my_dao_package/

### Upload to PyPI (Optional)

If you want to share the package publicly or more broadly, you can publish it on PyPI. Here’s how to do it:

1. **Create a Distribution Package**  
   Ensure you have a distribution package created using `setuptools`:

   ```bash
   python setup.py sdist bdist_wheel


In [ ]:
# Build the package
python setup.py sdist bdist_wheel

# Install twine if you don't have it already
pip install twine

# Upload the package
twine upload dist/*


## 3. Use the Package in Other Projects

### Install the Package

In your new projects, you can install the package via `pip`. Here’s how to do it:

1. **Install from PyPI**  
   If your package is published on PyPI, install it using `pip`:

   ```bash
   pip install your-package-name


In [ ]:
pip install my_dao_package

### Use the DAO in Your New Project

#### Example Project Structure

Here’s an example of how you might structure a new project that uses the DAO package:



In [ ]:
new_project/
│
├── app/
│   └── main.py
│
├── models/
│   └── user.py
│
└── requirements.txt


### `app/main.py`

In [ ]:
from my_dao.session import get_session
from my_dao.user_dao import UserDAO
from models.user import User  # Assuming you have defined the User model similarly as before

def main():
    session = get_session()
    user_dao = UserDAO(session)
    
    # Create a new user
    new_user = User(name="John Doe", email="john@example.com")
    user_dao.create_user(new_user)
    
    # Fetch the user by id
    fetched_user = user_dao.get_user_by_id(new_user.id)
    print(f"User fetched: {fetched_user.name}")

if __name__ == '__main__':
    main()


With this setup, you've successfully modularized your DAO code into a reusable package, making it easy to include in other projects.